In [ ]:
!pip install -q ultralytics

from google.colab import files
import cv2
import numpy as np
from ultralytics import YOLO

# 上传视频
uploaded = files.upload()
VIDEO_PATH = list(uploaded.keys())[0]
print("✔ 视频已上传:", VIDEO_PATH)


Saving 0a4bbedbd9a6c4e44779ee753349b73d.mp4 to 0a4bbedbd9a6c4e44779ee753349b73d.mp4
✔ 视频已上传: 0a4bbedbd9a6c4e44779ee753349b73d.mp4


In [ ]:
# ===== 参数，可以调 =====
OUTPUT_NAME   = "follow_players_smoother.mp4"
CROP_RATIO    = 0.6    # 中心裁剪比例（0.6 越小越“跟人”）
SMOOTH_FACTOR = 0.9    # 平滑系数：0.9 非常“稳”，0.7 更“敏捷但更晃”
MIN_CONF      = 0.4    # YOLO 置信度阈值
MIN_PEOPLE    = 3      # 至少检测到多少个人才更新中心，防止因为漏检乱跳
MAX_SHIFT_RATE = 0.03  # 每帧最多移动画面宽/高的 3%（防止瞬间跳）

# ===== 模型 & 视频信息 =====
model = YOLO("yolov8n.pt")

cap = cv2.VideoCapture(VIDEO_PATH)
fps = cap.get(cv2.CAP_PROP_FPS)
W  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
H  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

print(f"原始视频分辨率: {W} x {H}, FPS = {fps}")

crop_w = int(W * CROP_RATIO)
crop_h = int(H * CROP_RATIO)

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(OUTPUT_NAME, fourcc, fps, (crop_w, crop_h))

# 初始中心 = 画面中心
center_x = W / 2
center_y = H / 2

max_shift_x = W * MAX_SHIFT_RATE
max_shift_y = H * MAX_SHIFT_RATE

frame_id = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_id += 1

    # ---------- 1. YOLO 检测人 ----------
    results = model(frame, verbose=False)
    boxes = results[0].boxes

    person_centers = []
    for box in boxes:
        cls_id = int(box.cls[0].item())
        conf = float(box.conf[0].item())
        if cls_id != 0:      # 只要 person
            continue
        if conf < MIN_CONF:
            continue
        x1, y1, x2, y2 = box.xyxy[0].tolist()
        cx = (x1 + x2) / 2
        cy = (y1 + y2) / 2
        person_centers.append((cx, cy))

    # ---------- 2. 计算目标中心，但加限制 ----------
    if len(person_centers) >= MIN_PEOPLE:
        centers_np = np.array(person_centers)
        raw_target_cx = centers_np[:, 0].mean()
        raw_target_cy = centers_np[:, 1].mean()

        # 平滑：中心不会立刻跳到目标
        new_center_x = SMOOTH_FACTOR * center_x + (1 - SMOOTH_FACTOR) * raw_target_cx
        new_center_y = SMOOTH_FACTOR * center_y + (1 - SMOOTH_FACTOR) * raw_target_cy

        # 限制单帧移动距离，防止突然大跳
        dx = new_center_x - center_x
        dy = new_center_y - center_y
        dx = np.clip(dx, -max_shift_x, max_shift_x)
        dy = np.clip(dy, -max_shift_y, max_shift_y)

        center_x += dx
        center_y += dy
        # 如果检测到人太少，就保持原中心不动（用上一帧）
    else:
        pass  # 不更新 center_x, center_y

    # ---------- 3. 根据中心裁剪 ----------
    x0 = int(center_x - crop_w / 2)
    y0 = int(center_y - crop_h / 2)

    # 边界处理
    x0 = max(0, min(x0, W - crop_w))
    y0 = max(0, min(y0, H - crop_h))

    crop = frame[y0:y0+crop_h, x0:x0+crop_w]
    out.write(crop)

    if frame_id % 50 == 0:
        print(f"帧 {frame_id}, center=({center_x:.1f},{center_y:.1f}), 检测人数={len(person_centers)}")

cap.release()
out.release()



原始视频分辨率: 368 x 640, FPS = 24.0
帧 50, center=(179.5,317.1), 检测人数=2
帧 100, center=(213.8,337.7), 检测人数=4


In [ ]:
from google.colab import files
files.download(OUTPUT_NAME)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>